## Analyse des Métadonnées

Le catalogue Ecosphères a pour vocation de faciliter l'identification sur chaque territoire des données utiles à l'appui des politiques publiques (diagnostic, planificaiton, suivi). Dans le cadre de ces observations, nous nous intéressons au potentiel de priorisation des données remontées lors d'une recherche à partir des métadonnées disponibles au regard de critères thématiques, géographiques, temporels, d'accès à la donnée et enfin de fiabilité. Nous nous appuierons sur une sélection de métadonnées parmi les champs décrits dans des documents de références tels que la documentation du profile [GeoDCAT-AP](https://semiceu.github.io/GeoDCAT-AP/releases/) et le guide de saisie des éléments de métadonnées [INSPIRE](https://semiceu.github.io/GeoDCAT-AP/releases/).

Les métadonnées recueillies par le POC CKAN Ecosphères à partir des [points de moissonnage](https://github.com/ecolabdata/guichetdonnees-public/blob/main/moissonnages.json) sont à la base de ces observations. Les métadonnées sur les jeux de données sont entreposés dans les tables *package* et *package_extra* de la [base de données CKAN](https://boykoc.github.io/ckan/2019/10/21/ckan-283-database-diagram.html), également disponibles par API suivant le [schéma Ecosphères](https://github.com/ecolabdata/ckanext-ecospheres/blob/main/ckanext/ecospheres/scheming/ecospheres_dataset_schema.yaml) DCAT compatible.

In [ ]:
import pandas as pd
from datetime import datetime
from IPython.display import Markdown as md
from utils import wrapper_engine

engine = wrapper_engine('config.ini')

# Load data
with engine.connect() as connection:
    package = pd.read_sql_table(
        table_name="package", 
        con=connection,
        schema="public",
        )
    
    package_extra = pd.read_sql_table(
        table_name="package_extra", 
        con=connection,
        schema="public",
        )

<style type="text/css">
    /* Custom footer, <div> type so positioned as desired. */
    #header_left {
        position: absolute;
        top: 0%;
        left: 2.5%;
        font-size: large;
        font-weight: bold;
    }

    #header_right {
        position: absolute;
        top: 0%;
        right: 2.5%;
        font-size: large;
        font-weight: bold;
    }
</style>

<div id="hidden" style="display:none;">
    <div id="header">
        <p id="header_left">UPDATE</p>
        <p id="header_right">UPDATE</p>
    </div>
</div>

<script src="https://code.jquery.com/jquery-2.2.4.min.js"></script>
<script type="text/javascript">
    var header = $('#header').html();
    if ( window.location.search.match( /print-pdf/gi ) ) {
        Reveal.addEventListener( 'ready', function( event ) {
            $('.slide-background').append(header);
        });
    }
    else {
        $('div.reveal').append(header);
   }
</script>

**Taux de remplissage**

In [ ]:
# See metadata: rights
empty_metadata = [
    '{"fr": [], "en": []}',
    '{}'
]

keys_to_drop = [
    'harvest_object_id',
    'harvest_source_id',
    'harvest_source_title'
]

fill_rate = package_extra[["key", "value", "package_id"]].copy(deep=True)

fill_rate["value"] = fill_rate["value"].apply(lambda x: None if x in empty_metadata else x)
fill_rate.dropna(inplace=True)

fill_rate = fill_rate[~fill_rate["key"].isin(keys_to_drop)]

fill_rate = fill_rate.groupby("key", as_index=False).count()
fill_rate["package_id"] = fill_rate["package_id"].apply(lambda x: x/len(package[package["type"]=='dataset'])*100)

In [ ]:
import plotly
import plotly.express as px

plotly.offline.init_notebook_mode(connected=True)
fig = px.bar(fill_rate.sort_values(by=["package_id", "key"], ascending=False),
             x='key', 
             y='package_id',
             color_discrete_sequence=['#000091'],
            labels={"key": "Métadonnée",
                    "package_id": "Remplissage [%]"})
fig.add_hline(
    y=int(fill_rate["package_id"].median()), 
    line_width=3, 
    line_dash="dash", 
    line_color="red",
    annotation_text="médiane",
    annotation_textangle = 0)
plotly.offline.iplot(fig)